<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/classification_eval_with_OpenAI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Classification template


In [1]:
OKAREO_API_KEY = "<YOUR-OKAREO-API-KEY>"


In [ ]:
# Which model should the classifier use?
# We currently support OpenAI models and Cohere models.
# If you have your own model, you can invoke it through our CustomModel class
OPENAI_API_KEY = "<YOUR-OPENAI-API-KEY>"
COHERE_API_KEY = "<YOUR-COHERE-API-KEY>"

In [ ]:
%pip install okareo

When using OpenAI, you can pass in a system prompt and a user prompt. These system prompt should explain what the system should do with the user prompt. The user prompt template allows you to customize the prompt for the "user" role.

In [2]:
# Simple adhoc classifier using OpenAI'a GPT 3.5 Turbo model
# These prompts will be supplied to OpenAI
USER_PROMPT_TEMPLATE = "Lorem ipsum {input} Lorem ipsum"

CLASSIFICATION_CONTEXT_TEMPLATE = """
You will be provided a question from a customer.
Classify the question into a customer category and sub-category.
Provide the output with only the category name.

Categories: Technical Support, Billing ...

Sub-Categories for Technical Support:
Troubleshooting
...
"""

Below you can see how a OpenAI, Cohere and Custom model is set up

In [ ]:

from okareo import Okareo
# import OpenAIModel class to register the model
from okareo.model_under_test import OpenAIModel, CohereModel, CustomModel

okareo = Okareo(OKAREO_API_KEY)

model_name = "your-model-name"

# Register your model so it can be evaluated on a scenario set
model_under_test = okareo.register_model(
        # name=f"AdHoc OpenAI Classifier - {random_string}",
        name=model_name,
        model=OpenAIModel(
            # Fill in the model id of your choosing, e.g. gpt-4-turbo-preview
            model_id="gpt-3.5-turbo",

            # Add the temperature of the model
            temperature=0, 
            system_prompt_template=CLASSIFICATION_CONTEXT_TEMPLATE,
            user_prompt_template=USER_PROMPT_TEMPLATE,
        ),
    )

class CustomClassificationModel(CustomModel):
    # Callable to be applied to each scenario in the scenario set
    # You can add an endpoint to call your model or call your model directly
    def invoke(self, input: str) -> tuple:
        # call your model being tested here using <input> from the scenario set
        if "how much" in input:
            actual = "pricing"
        elif "return" in input:
            actual = "returns"
        else:
            actual = "complaints"

        # return a tuple of (actual, overall model response context)
        return actual, {"labels": actual, "confidence": .99 }  

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(
    name=model_name, 
    model=CustomClassificationModel(name="custom classification model"))


Create a scenario to run the classifier on

In [4]:
# Run the evaluation on the model and get a link to the results in Okareo

from okareo import Okareo

from okareo_api_client.models.test_run_type import TestRunType
from okareo_api_client.models import ScenarioSetCreate, SeedData, ScenarioType


# Generate example scenario based on seed data
random_string = ''.join(random.choices(string.ascii_letters, k=5))
scenario_set_create = ScenarioSetCreate(
    name=f"OpenAI AdHoc Category - {random_string}",
    number_examples=1,
    generation_type=ScenarioType.SEED,
    seed_data=[
        SeedData(
            input_="Can I connect my CRM?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have a way to send marketing emails?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Can I get invoiced instead of using a credit card?", 
            result="Billing"
        ),
        SeedData(
            input_="My CRM integration is not working.", 
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have SOC II tpye 2 certification?", 
            result="Account Management"
        ),
        SeedData(
            input_="I like the product.  Please connect me to your enterprise team.", 
            result="General Inquiry"
        )
    ],
)
scenario = okareo.create_scenario_set(scenario_set_create)

# run the test and call the model for each item in the scenario set
evaluation = model_under_test.run_test(
        name=f"adhoc-class-run-{random_string}",
        scenario=scenario,
        api_key=OPENAI_API_KEY,
        test_run_type=TestRunType.MULTI_CLASS_CLASSIFICATION,
        calculate_metrics=True,
    )

print(evaluation.app_link)

https://app.okareo.com/project/9255bce6-8704-42f7-8e8d-867f13cd9328/eval/30568c66-faf4-4ceb-902a-9ab04912a790
